In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from mycnn import GoogLeNet

In [ ]:
cnn = GoogLeNet(
    classes_num=2,
    logdir="./log_inception",
    weights_path="./log_inception/weights.h5"
)

In [ ]:
#%% preporcessing image to 4D tensor
img_path = "datasets/DogsVsCats/train/Dogs/0.jpg"
assert os.path.exists(img_path) == True
img = image.load_img(img_path, target_size=(224,224), color_mode="rgb")
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0).astype('float32')/255.
print(img_tensor.shape)
plt.imshow(img_tensor[0])
plt.grid(color='r', linestyle='-', linewidth=1)
plt.show()

In [ ]:
cnn.show_featuremap(img_tensor)

## [DEV] search conv and pool layer

### [TODO] can enter block layer, and show FM in block layer

In [ ]:
FM_LIST = [tf.keras.layers.Conv2D, tf.keras.layers.MaxPool2D, tf.keras.layers.AveragePooling2D]

model = cnn.model

layer_outputs = []
for layer in model.layers:
    if layer.__class__ in FM_LIST:
        print(layer.__class__)
        op = layer.output
        if len(op.shape) == 4:
            print(op.shape)
            layer_outputs.append(op)

activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_tensor)
print(len(activations))
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
from PIL import Image

layer_names = []

for layer in model.layers[1:5]:
    layer_names.append(layer.name)

images_per_row = 2

cnt = 0
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]

    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
    scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    plt.show()
    savefig = np.abs(display_grid)/np.abs(np.max(display_grid)) * 255

    cnt += 1

In [21]:
from mycnn.googlenet import InceptionV1

InceptionV1.__base__ == tf.keras.Model

tf.keras.layers.ELU.__base__

keras.engine.base_layer.Layer